In [1]:
#!/usr/bin/env bash

<span style="color: red; font-weight: bold">Use the following command to launch a server for this notebook:</span>

```bash
git clone git@github.com:input-output-hk/marlowe-cardano.git
cd marlowe-cardano/marlowe-runtime/
nix run ../marlowe-cli
```

Then navigate to the `examples/` folder in Jupyter and open this notebook.

# Demonstrating the Marlowe Transaction Deposit Component of Marlowe Runtime Using Role Tokens

## Preliminaries

Record version numbers

In [2]:
marlowe-cli --version

marlowe-cli 0.0.9.0


In [3]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [4]:
git rev-parse HEAD

afc61675c8223fc7ed26d7449d442b623562e0c4


### Setup the faucet.

Set the location of keys.

In [5]:
TREASURY=treasury

Set the faucet.

In [6]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDR"

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Select network

In a separate terminal, set up a tunnel to the Marlowe Runtime development server:
```bash
rm /tmp/preview.socket
ssh -NT \
  -L/tmp/preview.socket:/data/networks/preview/node.socket \
  -L 3717:127.0.0.1:23717 \
  -L 3718:127.0.0.1:23718 \
  -L 3719:127.0.0.1:23719 \
  -L 3721:127.0.0.1:23721 \
  -L 3723:127.0.0.1:23723 \
  54.202.238.5                                              
```

In [7]:
export CARDANO_NODE_SOCKET_PATH=/tmp/preview.socket
export CARDANO_TESTNET_MAGIC=2
MAGIC=(--testnet-magic 2)
echo "${MAGIC[@]}"

--testnet-magic 2


### Check that the reference script has been published

Check that the Marlowe semantics validator was published.

In [8]:
marlowe-cli transaction find-published


Searching for reference script at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"
{
    "marlowe": {
        "hash": "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#1"
    },
    "payout": {
        "hash": "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#2"
    }
}


### Participants

#### The Party

In [9]:
PARTY_SKEY="$TREASURY/john-fletcher.skey"
PARTY_VKEY="$TREASURY/john-fletcher.vkey"

Create the first party's keys, if necessary.

In [10]:
if [[ ! -e "$PARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_SKEY" --verification-key-file "$PARTY_VKEY"
fi
PARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$PARTY_VKEY")
echo "$PARTY_ADDR"

addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r


Fund the address.

In [11]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600 \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$PARTY_ADDR"

TxId "593ab47aea303e68eb0ce01dee8ed15da74477f304eb7ce32e354aed5eaaccc4"


#### The Counterparty

In [12]:
COUNTERPARTY_SKEY="$TREASURY/thomas-kyd.skey"
COUNTERPARTY_VKEY="$TREASURY/thomas-kyd.vkey"

Create the second party's keys, if necessary.

In [13]:
if [[ ! -e "$COUNTERPARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$COUNTERPARTY_SKEY" --verification-key-file "$COUNTERPARTY_VKEY"
fi
COUNTERPARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$COUNTERPARTY_VKEY")
echo "$COUNTERPARTY_ADDR"

addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5


Fund the address.

In [14]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600  \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$COUNTERPARTY_ADDR"

TxId "8f28d4e8abd56078fb00590cd4b60cba071884a0ea2d04ad672c118f3cf73351"


### Time computations

In [15]:
SECOND=1000
MINUTE=$((60 * SECOND))

In [16]:
NOW="$(($(date -u +%s) * SECOND))"
echo "$NOW"

1666269855000


## The Contract

We set the parameters for the ACTUS PAM contract.

In [17]:
MINIMUM_ADA=3000000

FIXED_POINT=1000000
PRINCIPAL=100
INTEREST_RATE=0.02
INTEREST=$(jq -n $PRINCIPAL*$INTEREST_RATE)

STATUS_DATE=$(date -d "$(date -u -R -d @$((NOW/1000)))" +"%Y-%m-%dT00:00:00")
INITIAL_EXCHANGE_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 1 year" +"%Y-01-01T00:00:00")
MATURITY_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 2 year" +"%Y-01-01T00:00:00")

In [18]:
yaml2json << EOI > deposit.actus
scheduleConfig:
  businessDayConvention: "NULL"
  endOfMonthConvention: "EOM"
  calendar: "NC"
maturityDate: "$MATURITY_DATE"
contractId: "0"
enableSettlement: false
initialExchangeDate: "$INITIAL_EXCHANGE_DATE"
contractRole: "RPA"
penaltyType: "O"
cycleAnchorDateOfInterestPayment: "$INITIAL_EXCHANGE_DATE"
contractType: "PAM"
notionalPrincipal: $PRINCIPAL
contractPerformance: "PF"
collateralAmount: 0
dayCountConvention: "30E360"
accruedInterest: 0
statusDate: "$STATUS_DATE"
cycleOfInterestPayment: "P1YL1"
prepaymentEffect: "N"
nominalInterestRate: $INTEREST_RATE
interestCalculationBase: "NT"
EOI
cat deposit.actus

{"accruedInterest":0,"collateralAmount":0,"contractId":"0","contractPerformance":"PF","contractRole":"RPA","contractType":"PAM","cycleAnchorDateOfInterestPayment":"2023-01-01T00:00:00","cycleOfInterestPayment":"P1YL1","dayCountConvention":"30E360","enableSettlement":false,"initialExchangeDate":"2023-01-01T00:00:00","interestCalculationBase":"NT","maturityDate":"2024-01-01T00:00:00","nominalInterestRate":0.02,"notionalPrincipal":100,"penaltyType":"O","prepaymentEffect":"N","scheduleConfig":{"businessDayConvention":"NULL","calendar":"NC","endOfMonthConvention":"EOM"},"statusDate":"2022-10-20T00:00:00"}


Create the contract.

In [19]:
marlowe-cli template actus \
  --minimum-ada "$MINIMUM_ADA" \
  --party Lender \
  --counter-party Borrower \
  --actus-terms-file  deposit.actus \
  --out-contract-file deposit-1.contract \
  --out-state-file /dev/null

Since we are testing, we don't really want to wait months or years for timeouts, so we edit the contract file to change the maturity date to 15 minutes from now and the initial deposit to 10 minutes from now.

In [ ]:
sed -i \
  -e "s/$(jq '.timeout' deposit-1.contract)/$((NOW + 10 * MINUTE))/" \
  -e "s/$(($(date -d "$MATURITY_DATE" -u +%s) * SECOND))/$((NOW + 15 * MINUTE))/" \
  deposit-1.contract

View the contract.

In [20]:
json2yaml deposit-1.contract

timeout: 1672531200000
timeout_continuation: close
when:
- case:
    deposits:
      negate:
        negate: 100000000
    into_account:
      role_token: Lender
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: Lender
  then:
    from_account:
      role_token: Lender
    pay: 100000000
    then:
      timeout: 1704067200000
      timeout_continuation: close
      when:
      - case:
          deposits: 2000000
          into_account:
            role_token: Borrower
          of_token:
            currency_symbol: ''
            token_name: ''
          party:
            role_token: Borrower
        then:
          from_account:
            role_token: Borrower
          pay: 2000000
          then:
            timeout: 1704067200000
            timeout_continuation: close
            when:
            - case:
                deposits: 100000000
                into_account:
                  role_token: Borrower
                of_token:
    

## Run the Contract

### Transaction 1. Create the contract

<span style="color:red;font-weight:bold">Create with role tokens fails:</span>

In [21]:
marlowe create \
  --core-file deposit-1.contract \
  --role "Lender=$PARTY_ADDR" \
  --role "Borrower=$COUNTERPARTY_ADDR" \
  --min-utxo "$MINIMUM_ADA" \
  --change-address "$PARTY_ADDR" \
  --address "$PARTY_ADDR" \
  --manual-sign deposit-1.txbody

CreateFailed (CreateBuildupFailed CollateralSelectionFailed)


: 1

<span style="color:red;font-weight:bold">However, there is collateral at the address:</span>

In [22]:
cardano-cli query utxo --testnet-magic 2 --address $PARTY_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
593ab47aea303e68eb0ce01dee8ed15da74477f304eb7ce32e354aed5eaaccc4     1        250000000 lovelace + TxOutDatumNone


<span style="color:red;font-weight:bold">Try specifying the collateral explicitly, even though we shouldn't have to:</span>

In [23]:
COLLATERAL=$(marlowe-cli util select --lovelace-only "$((5*ADA))" "$PARTY_ADDR" | sed -n -e '1{s/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;p}')
echo "$COLLATERAL"

593ab47aea303e68eb0ce01dee8ed15da74477f304eb7ce32e354aed5eaaccc4#1


<span style="color:red;font-weight:bold">Now we see a deserialization failure:</span>

In [24]:
marlowe create \
  --core-file deposit-1.contract \
  --role "Lender=$PARTY_ADDR" \
  --role "Borrower=$COUNTERPARTY_ADDR" \
  --min-utxo "$MINIMUM_ADA" \
  --collateral-utxo "$COLLATERAL" \
  --change-address "$PARTY_ADDR" \
  --address "$PARTY_ADDR" \
  --manual-sign deposit-1.txbody

Tx client client failure
marlowe: DeserializeError {message = "not enough bytes", offset = 0, unconsumedInput = ""}


: 1

<span style="color:red;font-weight:bold">We cannot proceed further.</span>

Build the transaction.

In [ ]:
TX_1=$(
marlowe create \
  --core-file deposit-1.contract \
  --role "Lender=$PARTY_ADDR" \
  --role "Borrower=$COUNTERPARTY_ADDR" \
  --min-utxo "$MINIMUM_ADA" \
  --collateral-utxo "$(marlowe-cli util select --lovelace-only "$((5*ADA))" "$PARTY_ADDR" | sed -n -e '1{s/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;p}')" \
  --change-address "$PARTY_ADDR" \
  --address "$PARTY_ADDR" \
  --manual-sign deposit-1.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
CONTRACT_ID="$TX_1"
echo "CONTRACT_ID = TX_1 = $CONTRACT_ID"

Sign the transaction.

In [ ]:
cardano-cli transaction sign \
  --tx-body-file deposit-1.txbody \
  --out-file     deposit-1.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [ ]:
marlowe submit deposit-1.tx

View the contract's UTxO.

In [ ]:
CONTRACT_ADDR=$(marlowe-cli contract address)
echo "$CONTRACT_ADDR"

In [ ]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_1//#*/}/p"

View the party's UTxOs.

In [ ]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

View the counterparty's UTxOs.

In [ ]:
cardano-cli query utxo "${MAGIC[@]}" --address "$COUNTERPARTY_ADDR"

Watch the contract.

In [ ]:
marlowe add "$CONTRACT_ID"

View the contract's history.

In [ ]:
marlowe log --show-contract "$CONTRACT_ID"

### Transaction 2. Party deposits loan amount

The party deposits the loan amount.

In [ ]:
TX_2=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$PARTY_ADDR" \
  --to-party "$PARTY_ADDR" \
  --lovelace "$((PRINCIPAL*FIXED_POINT))" \
  --validity-lower-bound "$((NOW-5*MINUTE))" \
  --validity-upper-bound "$((NOW+9*MINUTE))" \
  --change-address "$PARTY_ADDR" \
  --address "$PARTY_ADDR" \
  --manual-sign deposit-2.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_2 = $TX_2"

Sign the transaction.

In [ ]:
cardano-cli transaction sign \
  --tx-body-file deposit-2.txbody \
  --out-file     deposit-2.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [ ]:
marlowe submit deposit-2.tx

View the contract's UTxO.

In [ ]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_2//#*/}/p"

View the party's UTxOs.

In [ ]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

View the counterparty's UTxOs.

In [ ]:
cardano-cli query utxo "${MAGIC[@]}" --address "$COUNTERPARTY_ADDR"

View the contract's history.

In [ ]:
marlowe log --show-contract "$CONTRACT_ID"

### Transaction 3. Counterparty repays the loan's interest

The counterparty repays the interest.

In [ ]:
TX_3=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$COUNTERPARTY_ADDR" \
  --to-party "$COUNTERPARTY_ADDR" \
  --lovelace "$((INTEREST*FIXED_POINT))" \
  --validity-lower-bound "$((NOW-5*MINUTE))" \
  --validity-upper-bound "$((NOW+9*MINUTE))" \
  --change-address "$COUNTERPARTY_ADDR" \
  --address "$COUNTERPARTY_ADDR" \
  --manual-sign deposit-3.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_3 = $TX_3"

Sign the transaction.

In [ ]:
cardano-cli transaction sign \
  --tx-body-file deposit-3.txbody \
  --out-file     deposit-3.tx \
  --signing-key-file "$COUNTERPARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [ ]:
marlowe submit deposit-3.tx

View the contract's UTxO.

In [ ]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_3//#*/}/p"

View the party's UTxOs.

In [ ]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

View the counterparty's UTxOs.

In [ ]:
cardano-cli query utxo "${MAGIC[@]}" --address "$COUNTERPARTY_ADDR"

View the contract's history.

In [ ]:
marlowe log --show-contract "$CONTRACT_ID"

### Transaction 4. Counterparty repays the loan's principal

The counterparty repays the principal.

In [ ]:
TX_4=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$COUNTERPARTY_ADDR" \
  --to-party "$COUNTERPARTY_ADDR" \
  --lovelace "$((PRINCIPAL*FIXED_POINT))" \
  --validity-lower-bound "$((NOW-5*MINUTE))" \
  --validity-upper-bound "$((NOW+9*MINUTE))" \
  --change-address "$COUNTERPARTY_ADDR" \
  --address "$COUNTERPARTY_ADDR" \
  --manual-sign deposit-4.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_4 = $TX_4"

Sign the transaction.

In [ ]:
cardano-cli transaction sign \
  --tx-body-file deposit-4.txbody \
  --out-file     deposit-4.tx \
  --signing-key-file "$COUNTERPARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [ ]:
marlowe submit deposit-4.tx

Check that there is no UTxO for the contract.

In [ ]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_4//#*/}/p"

View the party's UTxOs.

In [ ]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

View the counterparty's UTxOs.

In [ ]:
cardano-cli query utxo "${MAGIC[@]}" --address "$COUNTERPARTY_ADDR"

View the contract's history.

In [ ]:
marlowe log --show-contract "$CONTRACT_ID"

## Cleanup

Remove the contract from history tracking.

In [ ]:
marlowe rm "$CONTRACT_ID"

In [ ]:
marlowe ls

Consolidate the UTxOs at the addresses.

In [25]:
marlowe-cli util clean \
  --change-address "$PARTY_ADDR" \
  --required-signer "$PARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "636ac50d2be3d84ec6720ca9d97f590db0d2c26ad48a3753db887dcdc57ce647"


In [26]:
marlowe-cli util clean \
  --change-address "$COUNTERPARTY_ADDR" \
  --required-signer "$COUNTERPARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "da9c0b7558ab00c53c6f95eb5b462dd1340de4f95a872a056426c10b8dfc21ca"


Send the funds back to the faucet.

In [27]:
marlowe-cli transaction simple \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$PARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$COUNTERPARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --required-signer "$PARTY_SKEY" \
  --required-signer "$COUNTERPARTY_SKEY" \
  --change-address "$FAUCET_ADDR" \
  --out-file /dev/null \
  --submit 600

TxId "93e7114026fbed4fb49242d90bc54c47b685dee6710c1c35c0b6436e3433e637"


See that the funds have been returned to the faucet.

In [28]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
